![ups logo](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Logo_Universidad_Polit%C3%A9cnica_Salesiana_del_Ecuador.png/640px-Logo_Universidad_Polit%C3%A9cnica_Salesiana_del_Ecuador.png)

# Aprendizaje Automático

# Practica 03

# Deep Learning con Redes Neuronales Convolucionales (CNN). Transfer Learning y Métodos Ensemble. Clasificación de Imágenes con Dataset BraTS-2024

### Autor: Diego Tapia

#### Instalación e importación de librerías

In [4]:
import nibabel as nib
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from skimage.transform import rotate
from skimage.util import montage
import os
from sklearn.model_selection import train_test_split
import keras
#import cv2
import tensorflow
import random
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import numpy as np
from keras.callbacks import CSVLogger
import keras.backend as K
import zipfile
import pandas as pd

print("librerías importadas !")

librerías importadas !


## Descripción del dataset

El conjunto de datos de entrenamiento proporcionado para el desafío BraTS-GoAT-2024 consiste en 2252 exploraciones de mpMRI cerebrales junto con anotaciones de segmentación de regiones tumorales. Los volúmenes 3D fueron despojados del cráneo y re-muestreados a una resolución isotrópica de 1 mm, con dimensiones de (240, 240, 155) voxels. 

Todos los escaneos mpMRI de BraTS están disponibles como archivos NIfTI (.nii.gz) y se clasifican de las siguiente forma:

* t1n: imágenes nativas 
* t1c: imágenes ponderadas en T1 post-contraste (T1Gd),
* t2w: imágenes ponderadas en T2 
* t2f: volúmenes de inversión de recuperación atenuada de fluido en T2 (T2-FLAIR), 

T1, T2 y FLAIR son secuencias o tipos específicos de imágenes que se adquieren durante un escaneo de resonancia magnética. Cada una de estas secuencias proporciona información única sobre los tejidos y las estructuras del cuerpo, y se utilizan para diferentes propósitos en el diagnóstico médico. 

T1 (Tiempo de Repetición 1):
Las imágenes T1 son sensibles a la cantidad de protones de hidrógeno presente en los tejidos y cómo están organizados. En estas imágenes, los tejidos con alto contenido de grasa aparecen brillantes, mientras que los tejidos con alto contenido de agua, como el líquido cerebroespinal, aparecen oscuros. Las imágenes T1 son útiles para resaltar la anatomía estructural, como el cerebro, los músculos y la médula espinal. También son útiles para evaluar la presencia de lesiones hemorrágicas, ya que la sangre también aparece brillante en estas imágenes.

T2 (Tiempo de Repetición 2):
Las imágenes T2 son sensibles a cómo interactúan los protones de hidrógeno entre sí y con su entorno. En estas imágenes, los tejidos con alto contenido de agua aparecen brillantes, mientras que los tejidos con bajo contenido de agua, como la grasa, aparecen oscuros. Las imágenes T2 son excelentes para detectar edema, inflamación y cambios patológicos en los tejidos. Son útiles en la evaluación de patologías como la esclerosis múltiple, tumores cerebrales y lesiones de la médula espinal.

FLAIR (Fluid Attenuated Inversion Recovery):
Las imágenes FLAIR son una variante de las imágenes T2 que suprimen la señal del líquido cerebroespinal (CSF, por sus siglas en inglés) para mejorar la visualización de las estructuras periventriculares y subcorticales del cerebro. Al suprimir la señal del CSF, las imágenes FLAIR pueden mejorar la detección de lesiones cerebrales y proporcionar una mejor diferenciación entre la sustancia blanca y la sustancia gris en el cerebro. Son útiles en el diagnóstico de enfermedades como la esclerosis múltiple, la encefalitis y las lesiones cerebrales traumáticas.

Todas estas imagenes fueron adquiridas con diferentes protocolos clínicos de múltiples instituciones contribuyentes de datos.

#### Segmentación

Todas las imágenes han sido anotadas manualmente por uno a cuatro evaluadores, siguiendo el mismo protocolo de anotación, y sus anotaciones fueron aprobadas por neurorradiólogos experimentados.

Las anotaciones para la segmentacion consisten en cuatro clases: 

* 1 para el núcleo tumoral necrótico (NCR), 
* 2 para el tejido edematoso peritumoral (ED), 
* 3 para el tumor realzante (ET) y 
* 0 para el fondo (voxels que no son parte del tumor).


## Análisis Exploratorio


Como se menciona anteriormente el dataset Brats cuenta con **2252** muestras de tomografías cerebrales a su vez cada muestra contiene 5 imágenes volumétricas correspondientes al tipo de mri t1n, t1c, t2w, t2f y segmentación, dando como resultado un total de **11260** imágenes volumétricas